In [ ]:
# Quick test
texts = [
    "I love this product!",
    "This is the worst experience ever.",
    "It's okay, nothing special."
]

results = sentiment_pipeline(texts, top_k=1)
for text, res in zip(texts, results):
    print(f"{text} -> {res[0]['label']} ({res[0]['score']:.3f})")

In [ ]:
# Initialize RoBERTa sentiment pipeline using CardiffNLP model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Build a TextClassificationPipeline
sentiment_pipeline = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    framework="pt",
    device=0 if torch.cuda.is_available() else -1
)

labels = model.config.id2label
labels

In [ ]:
# Optional: install required packages (uncomment if needed)
## Note: Running installs from the notebook may require internet access.
## On Windows cmd, you can also install via terminal:
## pip install --upgrade pip
## pip install transformers torch sentencepiece

# If you prefer inline install, uncomment the following:
# import sys
# !{sys.executable} -m pip install -U pip
# !{sys.executable} -m pip install transformers torch sentencepiece

# RoBERTa Sentiment Analysis
Setup environment and initialize a RoBERTa-based sentiment pipeline.